# ***BNLEARN DATASET***

In [ ]:
# bayesian dataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from pgmpy.utils import get_example_model
from pgmpy.sampling import BayesianModelSampling
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.preprocessing import LabelEncoder

alarm_model = get_example_model("cancer")
samples = BayesianModelSampling(alarm_model).rejection_sample(size=20000)
G = alarm_model
samples.head()
samples

In [ ]:
# plot bayesian dataset

plt.figure(figsize=(12, 8))
pos = nx.shell_layout(G)
nx.draw(G, pos, with_labels=True, node_color='lightblue', font_weight='bold',
        node_size=2000, font_size=12, arrowsize=20)
plt.title("Causal Graph using PC Algorithm")
plt.show()

# ***DATA VISUALIZATION***

In [ ]:
columns = samples.columns

fig, axes = plt.subplots(1, len(columns), figsize=(50, 5), constrained_layout=True)

for i, col in enumerate(columns):
    ax = axes[i]
    samples[col].value_counts().plot(kind='bar', ax=ax, color='skyblue', alpha=0.7)
    ax.set_title(f"Distribution of {col}")
    ax.set_xlabel(col)
    ax.set_ylabel("Frequency")

plt.show()

# ***PREPROCESSING***

In [ ]:
# Removing underscores from G model nodes
new_nodes = {node: node.replace('_', '') for node in G.nodes()}
G = nx.relabel_nodes(G, new_nodes)

print("Updated nodes:", G.nodes())

In [ ]:
# Removing underscores from Samples dataset nodes
samples.columns = samples.columns.str.replace('_', '')
samples

In [ ]:
# sample size
size_sample = samples.sample(n=20000, random_state=1)

In [ ]:
# Initialize LabelEncoder
le = LabelEncoder()

label_encoded_samples = size_sample.copy()
for column in label_encoded_samples.columns:
    if label_encoded_samples[column].dtype == 'object':
        label_encoded_samples[column] = le.fit_transform(label_encoded_samples[column])

label_encoded_samples.head()

In [ ]:
label_encoded_samples.columns.tolist()

In [ ]:
# unique values in each column
max_values = label_encoded_samples.max()+1
max_values

In [ ]:
# Flatten the entire dataset and calculate the overall max
overall_max = label_encoded_samples.values.flatten().max()
print(overall_max+1)

In [ ]:
# number of variables have single value
unique_counts = label_encoded_samples.nunique()

single_unique_value_columns = unique_counts[unique_counts == 1]


count_single_unique_value_columns = single_unique_value_columns.count()

print(f'Number of columns with a single unique value: {count_single_unique_value_columns}')


# **PC algoritm**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from pgmpy.estimators import PC
import time
start_time = time.time()

pc = PC(size_sample)
model = pc.estimate(variant='orig',return_type='dag', significance_level=0.01,ci_test='chi_square')
end_time = time.time()

execution_time_1 = end_time - start_time
print("Time :",execution_time_1)

In [ ]:

nodes_difference_2 = abs(len(G.nodes) - len(model.nodes))

# because we are not considering directions
ground_truth_edges = {tuple(sorted(edge)) for edge in G.edges()}
inferred_edges = {tuple(sorted(edge)) for edge in model.edges()}

false_positives = len(inferred_edges - ground_truth_edges)
false_negatives = len(ground_truth_edges - inferred_edges)
structural_hamming_distance_2 = false_positives + false_negatives


missing_edges_count_2 = 0
wrong_edges_count_2 = 0
wrong_directed_edges_count_2 = 0

for edge in ground_truth_edges:
    if edge not in inferred_edges:
        missing_edges_count_2 += 1

for edge in inferred_edges:
    if edge not in ground_truth_edges:
        wrong_edges_count_2 += 1
        
print("Time :",execution_time_1)
print("nodes difference", nodes_difference_2)
print("Number of missing edges:", missing_edges_count_2)
print("Number of wrong edges:", wrong_edges_count_2)
print("Structural Hamming Distance:", structural_hamming_distance_2)

# True positives are the edges that are both in ground truth and inferred edges
true_positives = len(ground_truth_edges & inferred_edges)

# Precision 
precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0

# Recall 
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

# F1 Score 
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Accuracy
accuracy = true_positives / (true_positives + false_positives + false_negatives) if (true_positives + false_positives + false_negatives) > 0 else 0

# Printing the metrics
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")
print(f"Accuracy: {accuracy:.4f}")


# ***TSETLIN MACHINE***

In [ ]:
#one hot-encoding
one_encoded_samples = pd.get_dummies(size_sample).astype(int)
one_encoded_samples

In [ ]:
# Multiclass
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import numpy as np
from tmu.models.classification.vanilla_classifier import TMClassifier
import time
from collections import defaultdict

one_hot_columns = one_encoded_samples.columns.tolist()
label_columns = label_encoded_samples.columns.tolist()
output_arrays = []
output_arrays_1 = []
start_time = time.time()

for target_column in label_columns:
    related_columns = [col for col in one_hot_columns if target_column.lower() in col.lower()]

    X = one_encoded_samples.drop(columns=related_columns)
    y = label_encoded_samples[target_column]

    samples = pd.concat([X, y], axis=1)

    class_counts = samples[target_column].value_counts()

    majority_class_size = class_counts.max()

    resampled_samples = []
    for class_value, count in class_counts.items():
        class_samples = samples[samples[target_column] == class_value]
        
        if count < majority_class_size:
            class_samples_upsampled = resample(class_samples,
                                               replace=True,  
                                               n_samples=majority_class_size,  
                                               random_state=42)
            resampled_samples.append(class_samples_upsampled)
        else:
            resampled_samples.append(class_samples)

    balanced_samples = pd.concat(resampled_samples)
    balanced_samples = balanced_samples.sample(n=len(size_sample), random_state=1)


    X_balanced = balanced_samples.drop(columns=[target_column])
    y_balanced = balanced_samples[target_column]

    X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

    X_train = X_train.astype(np.uint32)
    y_train = y_train.astype(np.uint32)
    
    
    tm = TMClassifier(
        number_of_clauses=2,  
        T=10, # Threshold
        s=1, # specificaity
        platform="CPU",
        weighted_clauses=True,
    )

    learned_clauses_all_epochs = []  

    for epoch in range(2):
        tm.fit(X_train.to_numpy(), y_train.to_numpy())

        if (epoch + 1) % 2 == 0:
        
            # as per the number of clauses used
            for i in range(2):
            # as per unique values in each column
                for j in range(2):
                        learned_clauses = tm.clause_banks[j].get_literals()[i][:len(X.columns) * 2] 
                        relevant_features = [feature for feature, clause in zip(X.columns.tolist() * 2, learned_clauses) if clause != 0]
                        output_variable = target_column
                        output_arrays.append([*relevant_features, output_variable])

                        weights_of_learned_clauses = tm.weight_banks[j].get_weights()[i] 
                        output_arrays_1.append([*relevant_features, output_variable, weights_of_learned_clauses])    
            
        result = 100 * (tm.predict(X_test.to_numpy()) == y_test.to_numpy()).mean()

        print(f"Target Column: {target_column}")
        print(f"Accuracy: {result:.2f}%\n")

end_time = time.time()
time_1 = end_time - start_time
print(f"Total execution time: {time_1} seconds")    

In [ ]:
# the last element in every list before wight number is the output variable and remaining are literals for that specific clause
output_arrays_1

In [ ]:
# Remove Suffix and Duplicates from lists (i.e clauses)
start_time = time.time()
def clean_and_deduplicate(arr):
    cleaned = []
    for item in arr:
        cleaned.append(item.split('_')[0])  
    
    cleaned = list(set(cleaned))
    return cleaned

cleaned_output = [clean_and_deduplicate(sublist[:-2]) + [sublist[-2], abs(sublist[-1])] for sublist in output_arrays_1]

print(cleaned_output)

end_time = time.time()
time_2 = end_time - start_time

In [ ]:
# Calculate Top variables are for each output element

start_time = time.time()

def calculate_input_output_frequency_weightage(cleaned_output, top_n):
    output_data = defaultdict(lambda: defaultdict(lambda: {"frequency": 0, "total_weightage": 0}))
    
    for entry in cleaned_output:
        *inputs, output, weight = entry
        
        for input_element in inputs:
            output_data[output][input_element]["frequency"] += 1
            output_data[output][input_element]["total_weightage"] += weight
    
    #print("Frequency and Weightage for each Input-Output Pair:")
    for output, input_data in output_data.items():
        for input_element, values in input_data.items():
            frequency = values["frequency"]
            total_weightage = values["total_weightage"]
            #print(f"Output: {output}, Input: {input_element}, Frequency: {frequency}, Total Weightage: {total_weightage} = {frequency + total_weightage}")

    result = []

    for output, input_data in output_data.items():
        sorted_inputs = sorted(input_data.items(), key=lambda x: x[1]["frequency"] * x[1]["total_weightage"], reverse=True)
        
        top_inputs = sorted_inputs[:top_n]

        result.append([input for input, _ in top_inputs] + [output])

    return result

top_n = 5
output = calculate_input_output_frequency_weightage(cleaned_output, top_n)
print("\nTop N Input-Output List:")
print(output)
unique_array = output

end_time = time.time()
time_3 = end_time - start_time

In [ ]:
start_time = time.time()

result = {}

for arr in unique_array:
    output_element = arr[-1]  
    input_elements = arr[:-1]  

    
    for input_element in input_elements:
        
        pair = (output_element, input_element)
        reverse_pair = (input_element, output_element)  
        
        
        if reverse_pair in result:
            continue
        
        remaining_elements = [el for el in arr if el != input_element and el != output_element]
        
        
        for next_arr in unique_array:
            if next_arr[-1] == input_element:
                
                next_remaining_elements = [el for el in next_arr if el != input_element and el != next_arr[-1]]

                
                combined_remaining_elements = set(remaining_elements + next_remaining_elements)
                combined_remaining_elements.discard(output_element)  
                combined_remaining_elements.discard(input_element)  

                result[pair] = list(combined_remaining_elements)

for key, value in result.items():
    print(f"Pair: {key}, Remaining elements: {value}")


end_time = time.time()
time_4 = end_time - start_time


In [ ]:
# whole testing
import matplotlib.pyplot as plt
from pgmpy.estimators import CITests
import networkx as nx
import pandas as pd
import itertools

start_time = time.time()
K = nx.Graph()
separating_sets = {}

for pair, Z in result.items():
    X, Y = pair
    
    chi_square_test = CITests.chi_square(X=X, Y=Y, Z=[], data=label_encoded_samples, boolean=True, significance_level=0.05)
    
    if not chi_square_test:  
        print(f"{X} and {Y} are not independent. Testing conditional independence with Z={Z}.")
        
        if Z:
            independent = False  
            for r in range(1, len(Z) + 1):  
                for z_combination in itertools.combinations(Z, r):
                    chi_square_test_with_Z = CITests.chi_square(X=X, Y=Y, Z=list(z_combination), data=label_encoded_samples, boolean=True, significance_level=0.05)
                    
                    if chi_square_test_with_Z:  
                        print(f"{X} and {Y} are conditionally independent with Z={z_combination}. No edge added.")
                        independent = True
                        break
            if not independent:
                    print(f"{X} and {Y} are not conditionally independent with Z={Z}. Adding edge to graph.")
                    K.add_edge(X, Y)
            else:
                    print(f"{X} and {Y} are conditionally independent with Z={Z}. No edge added.")
    else:
        print(f"{X} and {Y} are independent. No edge added.")
        for z_element in Z:
            chi_square_test_with_z_element = CITests.chi_square(X=X, Y=Y, Z=[z_element], data=label_encoded_samples, boolean=True, significance_level=0.05)
        
            if not chi_square_test_with_z_element:  
                print(f"{X} and {Y} become dependent given Z={z_element}. Adding edges.")
                if (X, Y) not in separating_sets:
                    separating_sets[(X, Y)] = set()
                separating_sets[(X, Y)].add(z_element)
            else:
                print(f"{X} and {Y} remain independent given Z={z_element}. No edge added.")
        
        

pos = nx.shell_layout(K)  
plt.figure(figsize=(10, 8))
nx.draw(K, pos, with_labels=True, node_color='lightblue', node_size=3000, font_size=15, font_weight='bold', arrowsize=20)
plt.title('Dependency Graph')
plt.show()

end_time = time.time()
time_5 = end_time - start_time

In [ ]:
nodes_difference_1 = abs(len(G.nodes) - len(K.nodes))

ground_truth_edges = {tuple(sorted(edge)) for edge in G.edges()}
inferred_edges = {tuple(sorted(edge)) for edge in K.edges()}

false_positives = len(inferred_edges - ground_truth_edges)
false_negatives = len(ground_truth_edges - inferred_edges)
structural_hamming_distance_1 = false_positives + false_negatives


missing_edges_count_1 = 0
wrong_edges_count_1 = 0
wrong_directed_edges_count_1 = 0

for edge in ground_truth_edges:
    if edge not in inferred_edges:
        missing_edges_count_1 += 1

for edge in inferred_edges:
    if edge not in ground_truth_edges:
        wrong_edges_count_1 += 1
        
print("Time", time_1+time_2+time_3+time_4+time_5)        
print("Nodes difference", nodes_difference_1)
print("Number of missing edges:", missing_edges_count_1)
print("Number of wrong edges:", wrong_edges_count_1)
print("Structural Hamming Distance:", structural_hamming_distance_1)

true_positives = len(ground_truth_edges & inferred_edges)

# Precision
precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0

# Recall
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

# F1 Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Accuracy
accuracy = true_positives / (true_positives + false_positives + false_negatives) if (true_positives + false_positives + false_negatives) > 0 else 0

# Printing the metrics
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")
print(f"Accuracy: {accuracy:.4f}")

In [ ]:
print("False positives (edges in K but not in G):", inferred_edges - ground_truth_edges)
print("False negatives (edges in G but not in K):", ground_truth_edges - inferred_edges)

In [ ]:
nodes_difference_2 = (G.nodes) - (K.nodes)
nodes_difference_2

# CHECK TEST PROVIDE WRONG

In [ ]:
# here we check the edges (i.e G but not in K), if found we find sepset for that edges if present means we did the tset but chi test provide they are independent 
ALL_EDGES =   ground_truth_edges - inferred_edges
ALL_EDGES

count = 0  

for key_to_find in ALL_EDGES:
    
    if key_to_find in result or (key_to_find[1], key_to_find[0]) in result:
        value = result.get(key_to_find, result.get((key_to_find[1], key_to_find[0])))
        print(f"Value associated with {key_to_find} or its reverse: {value}")
        count += 1  # Increase the count if the edge or its reverse is found
    #else:
        #print(f"Neither key found in the dictionary: {key_to_find}")

# Print the final count
print(f"Total count of found edges: {count}")

In [ ]:
# here we check the edges (i.e K but not in G), if found we find sepset for that edges if found sepset == required sepset, then test provide wrong.
ALL_EDGES =   inferred_edges - ground_truth_edges

# Initialize a counter for matches
match_count = 0

# Iterate over ALL_EDGES and check for matches
for key_to_find in ALL_EDGES:
    if key_to_find in result or (key_to_find[1], key_to_find[0]) in result:
        # Get value from result dictionary (either direct or reversed key)
        value = result.get(key_to_find, result.get((key_to_find[1], key_to_find[0])))
        print(f"Value associated with {key_to_find} or its reverse: {value}")
        
        # Get the minimal d-separating set for the edge
        start, end = key_to_find
        min_d_separator = G.minimal_dseparator(start=start, end=end)
        print(f"The minimal d-separating set between {start} and {end} is: {min_d_separator}")
        
        # Ensure both are sets for comparison
        min_d_separator_set = set(min_d_separator)
        value_set = set(value)
        
        # Check if min_d_separator is a subset of value_set (with the extra elements allowed in value)
        if min_d_separator_set.issubset(value_set):
            print(f"Match found for {key_to_find}: {min_d_separator_set} is a subset of {value_set}")
            # Increment the match count
            match_count += 1
        else:
            print(f"No match for {key_to_find}: {min_d_separator_set} is not a subset of {value_set}")
    else:
        print(f"Neither key found in the dictionary: {key_to_find}")

# Print the total match count
print(f"\nTotal matches: {match_count}")

# DIRECTED EDGES 

In [ ]:
# for colliders found
skeleton = set(K.edges())


directed_graph = set()

skeleton_1 = set(skeleton)

for (node1, node2), sep_set in separating_sets.items():
    for sep_node in sep_set:
        
        if (sep_node, node1) in skeleton or (node1, sep_node) in skeleton:
            directed_graph.add((node1, sep_node))  
            skeleton_1.discard((sep_node, node1))
            skeleton_1.discard((node1, sep_node))
        
        if (sep_node, node2) in skeleton or (node2, sep_node) in skeleton:
            directed_graph.add((node2, sep_node))  
            skeleton_1.discard((sep_node, node2))
            skeleton_1.discard((node2, sep_node))

print("Directed Graph:", directed_graph)
print("Remaining Skeleton_1:", skeleton_1)



In [ ]:
for edge in directed_graph:
    skeleton_1.discard(edge)
    skeleton_1.discard((edge[1], edge[0]))

queue = list(directed_graph) 

while queue:
    node1, node2 = queue.pop(0)  
    last_element = node2  
    
    to_remove = []
    for edge in skeleton_1:
        if last_element in edge:
            
            new_edge = (last_element, edge[0]) if edge[1] == last_element else (last_element, edge[1])
            directed_graph.add(new_edge)
            queue.append(new_edge)  
            to_remove.append(edge)
    
    for edge in to_remove:
        skeleton_1.discard(edge)
        
for edge in skeleton_1:
    directed_graph.add(edge)
print("Directed Graph:", directed_graph)
print("Remaining Skeleton_1:", skeleton_1)

In [ ]:
directed_graph = nx.DiGraph(directed_graph)

# Define the position layout (shell layout in this case)
pos = nx.shell_layout(G)

# Plot the graph
plt.figure(figsize=(10, 8))
nx.draw(directed_graph, pos, with_labels=True, node_color='lightblue', node_size=3000, font_size=15, font_weight='bold', arrowsize=20)
plt.title('Dependency Graph')
plt.show()


In [ ]:
nodes_difference_3 = abs(len(G.nodes) - len(directed_graph.nodes))
print("nodes difference", nodes_difference_3)

ground_truth_edges = set(G.edges())
inferred_edges = set(directed_graph.edges())

false_positives = len(inferred_edges - ground_truth_edges)
false_negatives = len(ground_truth_edges - inferred_edges)
structural_hamming_distance_3 = false_positives + false_negatives

print("Structural Hamming Distance:", structural_hamming_distance_3)

missing_edges_count_3 = 0
wrong_edges_count_3 = 0
wrong_directed_edges_count_3 = 0


for edge in ground_truth_edges:
    if edge not in inferred_edges and (edge[1], edge[0]) not in inferred_edges:
        missing_edges_count_3 += 1


for edge in inferred_edges:
    if edge not in ground_truth_edges and (edge[1], edge[0]) not in ground_truth_edges:
        wrong_edges_count_3 += 1
    elif edge not in ground_truth_edges and (edge[1], edge[0]) in ground_truth_edges:
        wrong_directed_edges_count_3 += 1


print("Number of missing edges:", missing_edges_count_3)
print("Number of wrong edges:", wrong_edges_count_3)


print("Number of wrong direction edges:", wrong_directed_edges_count_3)

true_positives = len(ground_truth_edges & inferred_edges)

# Precision
precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0

# Recall
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

# F1 Score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Accuracy
accuracy = true_positives / (true_positives + false_positives + false_negatives) if (true_positives + false_positives + false_negatives) > 0 else 0

# Printing the metrics
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")
print(f"Accuracy: {accuracy:.4f}")